In [7]:
import os
import tempfile
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
import imageio.v2 as imageio

# === CONFIGURAÇÕES DO USUÁRIO ===
base_path = '/media/fernando/KINGSTON/Doutorado/2025/reduced/20250227'
subdirs = ['sparc4acs1', 'sparc4acs2', 'sparc4acs3', 'sparc4acs4']
objeto = 'txcol'
arquivo_alvo = f'{objeto}_proc.fits'

# Parâmetros dos círculos
raio = 15
cor1 = 'red'
cor2 = 'blue'
alpha = 0.5

# Tamanho do recorte ao redor do primeiro círculo (ponto de referência)
crop_width = 300
crop_height = 300

# Coordenadas iniciais para cada pasta: (x0_1, y0_1), (x0_2, y0_2)
coordenadas_iniciais = {
    'sparc4acs1': ((538, 517), (558, 479)),
    'sparc4acs2': ((515, 515), (534, 478)),
    'sparc4acs3': ((521, 507), (540, 471)),
    'sparc4acs4': ((535, 519), (542, 483)),
}

# === PROCESSAMENTO POR SUBPASTA ===
for subdir in subdirs:
    (x0_1, y0_1), (x0_2, y0_2) = coordenadas_iniciais[subdir]
    full_path = os.path.join(base_path, subdir)
    arquivos_proc = []

    if not os.path.isdir(full_path):
        print(f"Pasta não encontrada: {full_path}")
        continue

    # Coleta os arquivos que terminam com {objeto}_proc.fits
    for fname in sorted(os.listdir(full_path)):
        if fname.endswith(arquivo_alvo):
            arquivos_proc.append(os.path.join(full_path, fname))

    if not arquivos_proc:
        print(f"Nenhum arquivo encontrado em {subdir}")
        continue

    gif_output = f'{objeto}_{subdir}.gif'

    with tempfile.TemporaryDirectory() as tmpdir:
        frame_paths = []

        for i, filepath in enumerate(arquivos_proc):
            with fits.open(filepath) as hdul:
                data = hdul[0].data
                header = hdul[0].header

                xshift = header.get('XSHIFT', 0)
                yshift = header.get('YSHIFT', 0)

                # Posições com shift
                x1 = x0_1 + xshift
                y1 = y0_1 + yshift
                x2 = x0_2 + xshift
                y2 = y0_2 + yshift

                # Recorte centrado no primeiro círculo
                x_min = int(max(x1 - crop_width // 2, 0))
                x_max = int(min(x1 + crop_width // 2, data.shape[1]))
                y_min = int(max(y1 - crop_height // 2, 0))
                y_max = int(min(y1 + crop_height // 2, data.shape[0]))
                data_crop = data[y_min:y_max, x_min:x_max]

                # Coordenadas relativas ao recorte
                x1_crop = x1 - x_min
                y1_crop = y1 - y_min
                x2_crop = x2 - x_min
                y2_crop = y2 - y_min

                # Criar figura e salvar frame temporário
                fig, ax = plt.subplots(figsize=(6, 6))
                im = ax.imshow(data_crop, cmap='gray', origin='lower',
                               vmin=np.percentile(data_crop, 5), vmax=np.percentile(data_crop, 99))

                # Círculo 1
                circ1 = Circle((x1_crop, y1_crop), raio, edgecolor=cor1,
                               facecolor='none', lw=2, alpha=alpha)
                ax.add_patch(circ1)

                # Círculo 2
                circ2 = Circle((x2_crop, y2_crop), raio, edgecolor=cor2,
                               facecolor='none', lw=2, alpha=alpha)
                ax.add_patch(circ2)

                ax.set_title(os.path.basename(filepath))
                ax.axis('off')

                frame_path = os.path.join(tmpdir, f'frame_{i:03d}.png')
                plt.savefig(frame_path, bbox_inches='tight', pad_inches=0)
                plt.close(fig)
                frame_paths.append(frame_path)

        # Criar o GIF final
        images = [imageio.imread(frame) for frame in frame_paths]
        imageio.mimsave(gif_output, images, duration=0.8, loop=0)
        print(f'GIF salvo como: {gif_output}')


ValueError: all input arrays must have the same shape

In [ ]:
from IPython.display import display, HTML

# Lista dos arquivos GIF gerados
gif_files = [
    f'{objeto}_sparc4acs1.gif',
    f'{objeto}_sparc4acs2.gif',
    f'{objeto}_sparc4acs3.gif',
    f'{objeto}_sparc4acs4.gif'
]

# HTML para exibir os GIFs em linha, com repetição infinita
html_code = '<div style="display: flex; gap: 10px;">'
for gif in gif_files:
    html_code += f'<div><img src="{gif}" style="width:300px;" loop="infinite" autoplay></div>'
html_code += '</div>'

display(HTML(html_code))

In [19]:
import os
from PIL import Image
from pathlib import Path

# Caminho da pasta onde estão os gifs
PASTA_ORIGEM = Path("/home/fernando/Desktop/gifs_workshop2025")
PASTA_SAIDA = PASTA_ORIGEM / "otimizados"
PASTA_SAIDA.mkdir(exist_ok=True)

# Função para otimizar o gif
def otimizar_gif(caminho_arquivo, caminho_saida, pular_frames=10):
    """
    pular_frames: use 1 para não pular, 2 para usar 1 frame a cada 2, etc.
    """
    img = Image.open(caminho_arquivo)
    frames = []
    durations = []

    index = 0
    try:
        while True:
            if index % pular_frames == 0:
                frame = img.copy()
                frames.append(frame)
                durations.append(img.info.get("duration", 40))  # duração padrão = 40ms
            index += 1
            img.seek(img.tell() + 1)
    except EOFError:
        pass

    if not frames:
        print(f"Nenhum frame foi salvo para {caminho_arquivo.name}")
        return

    # Salva com otimizações
    frames[0].save(
        caminho_saida,
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        loop=0,
        disposal=2,
        duration=durations
    )

# Processa todos os gifs
for arquivo in PASTA_ORIGEM.iterdir():
    if arquivo.suffix.lower() == ".gif":
        saida = PASTA_SAIDA / arquivo.name
        print(f"Otimizar: {arquivo.name}")
        otimizar_gif(
            caminho_arquivo=arquivo,
            caminho_saida=saida,
            pular_frames=1  # ou 2 se quiser reduzir a taxa de quadros pela metade
        )
        print(f"Salvo em: {saida}")


Otimizar: foaqrl4_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs2.gif
Otimizar: v2400ophl4_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs2.gif
Otimizar: v1025cen_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v1025cen_sparc4acs2.gif
Otimizar: v2400ophl4_sparc4acs1.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs1.gif
Otimizar: v2400ophl4_sparc4acs4.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs4.gif
Otimizar: foaqrl4_sparc4acs3.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs3.gif
Otimizar: foaqrl4_sparc4acs4.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs4.gif
Otimizar: foaqrl4_sparc4acs1.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs1.gif
Otimizar: v1025cen_sparc4acs1.gif
Sa

In [23]:
import os
from PIL import Image
from pathlib import Path

# Caminho da pasta onde estão os gifs
PASTA_ORIGEM = Path("/home/fernando/Desktop/gifs_workshop2025")
PASTA_SAIDA = PASTA_ORIGEM / "otimizados"
PASTA_SAIDA.mkdir(exist_ok=True)

# Função para otimizar o gif
def otimizar_gif(caminho_arquivo, caminho_saida, pular_frames=1, reduzir_resolucao=None, reduzir_cores=8):
    """
    pular_frames: use 1 para não pular, 2 para usar 1 frame a cada 2, etc.
    reduzir_resolucao: tupla (largura, altura), ex: (200, 200)
    reduzir_cores: número de cores para a paleta (geralmente 2 ou 4 para preto e branco)
    """
    img = Image.open(caminho_arquivo)
    frames = []
    durations = []

    index = 0
    try:
        while True:
            if index % pular_frames == 0:
                frame = img.copy()

                # Reduz a resolução, se necessário
                if reduzir_resolucao:
                    frame = frame.resize(reduzir_resolucao, Image.LANCZOS)
                
                # Reduz a paleta de cores para um número pequeno
                if reduzir_cores:
                    frame = frame.convert('P', palette=Image.ADAPTIVE, colors=reduzir_cores)

                frames.append(frame)
                durations.append(img.info.get("duration", 40))  # Duração padrão de 40ms
            index += 1
            img.seek(img.tell() + 1)
    except EOFError:
        pass

    if not frames:
        print(f"Nenhum frame foi salvo para {caminho_arquivo.name}")
        return

    # Salva o gif otimizado
    frames[0].save(
        caminho_saida,
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        loop=0,
        disposal=2,
        duration=durations
    )

# Processa todos os gifs
for arquivo in PASTA_ORIGEM.iterdir():
    if arquivo.suffix.lower() == ".gif":
        saida = PASTA_SAIDA / arquivo.name
        print(f"Otimizar: {arquivo.name}")
        otimizar_gif(
            caminho_arquivo=arquivo,
            caminho_saida=saida,
            pular_frames=2,  # Usando 1 frame a cada 2
            reduzir_resolucao=(200, 200),  # Reduzindo resolução para 200x200
            reduzir_cores=2  # Reduzindo para apenas 2 cores (preto e branco)
        )
        print(f"Salvo em: {saida}")


Otimizar: foaqrl4_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs2.gif
Otimizar: v2400ophl4_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs2.gif
Otimizar: v1025cen_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v1025cen_sparc4acs2.gif
Otimizar: v2400ophl4_sparc4acs1.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs1.gif
Otimizar: v2400ophl4_sparc4acs4.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs4.gif
Otimizar: foaqrl4_sparc4acs3.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs3.gif
Otimizar: foaqrl4_sparc4acs4.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs4.gif
Otimizar: foaqrl4_sparc4acs1.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs1.gif
Otimizar: v1025cen_sparc4acs1.gif
Sa

In [25]:
import os
from PIL import Image
from pathlib import Path

# Caminho da pasta onde estão os gifs
PASTA_ORIGEM = Path("/home/fernando/Desktop/gifs_workshop2025")
PASTA_SAIDA = PASTA_ORIGEM / "otimizados"
PASTA_SAIDA.mkdir(exist_ok=True)

# Função para otimizar o gif
def otimizar_gif(caminho_arquivo, caminho_saida, pular_frames=1, reduzir_resolucao=None):
    """
    pular_frames: use 1 para não pular, 2 para usar 1 frame a cada 2, etc.
    reduzir_resolucao: tupla (largura, altura), ex: (200, 200)
    """
    img = Image.open(caminho_arquivo)
    frames = []
    durations = []

    index = 0
    try:
        while True:
            if index % pular_frames == 0:
                frame = img.copy()

                # Reduz a resolução, se necessário
                if reduzir_resolucao:
                    frame = frame.resize(reduzir_resolucao, Image.LANCZOS)

                frames.append(frame)
                durations.append(img.info.get("duration", 40))  # Duração padrão de 40ms
            index += 1
            img.seek(img.tell() + 1)
    except EOFError:
        pass

    if not frames:
        print(f"Nenhum frame foi salvo para {caminho_arquivo.name}")
        return

    # Salva o gif otimizado
    frames[0].save(
        caminho_saida,
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        loop=0,
        disposal=2,
        duration=durations
    )

# Processa todos os gifs
for arquivo in PASTA_ORIGEM.iterdir():
    if arquivo.suffix.lower() == ".gif":
        saida = PASTA_SAIDA / arquivo.name
        print(f"Otimizar: {arquivo.name}")
        otimizar_gif(
            caminho_arquivo=arquivo,
            caminho_saida=saida,
            pular_frames=2,  # Usando 1 frame a cada 2
            reduzir_resolucao=(200, 200)  # Reduzindo resolução para 200x200
        )
        print(f"Salvo em: {saida}")


Otimizar: foaqrl4_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs2.gif
Otimizar: v2400ophl4_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs2.gif
Otimizar: v1025cen_sparc4acs2.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v1025cen_sparc4acs2.gif
Otimizar: v2400ophl4_sparc4acs1.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs1.gif
Otimizar: v2400ophl4_sparc4acs4.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/v2400ophl4_sparc4acs4.gif
Otimizar: foaqrl4_sparc4acs3.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs3.gif
Otimizar: foaqrl4_sparc4acs4.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs4.gif
Otimizar: foaqrl4_sparc4acs1.gif
Salvo em: /home/fernando/Desktop/gifs_workshop2025/otimizados/foaqrl4_sparc4acs1.gif
Otimizar: v1025cen_sparc4acs1.gif
Sa